In [1]:
import pandas as pd

In [2]:
import sklearn as sk

In [3]:
import numpy as np

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

In [5]:
df = pd.read_csv('data.csv')
df.head()
df = df.drop(['ca','cholesterol','oldpeak','slope','restecg','jenis_sakit_dada'],axis = 1)
dt = df.drop(['diagnosis'],axis = 1)
df.shape,dt.shape

((123, 8), (123, 7))

In [6]:
xTrain, xTest, yTrain, yTest = train_test_split(dt, df['diagnosis'], test_size = 0.25, random_state = 0)

In [7]:
xTrain.shape,yTrain.shape,xTest.shape,yTest.shape

((92, 7), (92,), (31, 7), (31,))

In [8]:
df.head()

,umur,jenis_kelamin,tekanan_darah,gula_darah,thalach,exang,thal,diagnosis
0,32,1,95,0,127,0,0,1
1,34,1,115,0,154,0,0,1
2,35,1,0,0,130,1,7,3
3,36,1,110,0,125,1,6,1
4,38,0,105,0,166,0,0,2


In [9]:
clf = svm.SVC(kernel='linear', C=1).fit(xTrain, yTrain)
clf.score(xTest, yTest)                           

0.2903225806451613